In [1]:
import enum
import cv2
import traitlets
import ipywidgets.widgets as widgets
import numpy as np

from camera import Camera, bgr8_to_jpeg

In [2]:
print("Reading calibration parameters ......")
cv_file = cv2.FileStorage("data/params_py.xml", cv2.FILE_STORAGE_READ)

Left_Stereo_Map_x = cv_file.getNode("Left_Stereo_Map_x").mat()
Left_Stereo_Map_y = cv_file.getNode("Left_Stereo_Map_y").mat()
Right_Stereo_Map_x = cv_file.getNode("Right_Stereo_Map_x").mat()
Right_Stereo_Map_y = cv_file.getNode("Right_Stereo_Map_y").mat()
cv_file.release()

Reading calibration parameters ......


In [40]:
cameraL = Camera.instance(id=0, width=820, height=616)
cameraR = Camera.instance(id=1, width=820, height=616)

In [41]:
wL=cameraL.widget(width=410, height=308)
wR=cameraR.widget(width=410, height=308)
display(widgets.HBox([wL,wR]))

In [5]:
def adjustGamma(image, gamma=1.0):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
    for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

In [37]:
def snapPhoto(b):
    global currentL, currentR
    
    currentL = cameraL.value
    currentR = cameraR.value
    dummy=makeAnaglyph(None) 
    
def makeAnaglyph(b):
    left = cv2.cvtColor(currentL, cv2.COLOR_BGR2GRAY)
    right = cv2.cvtColor(currentR, cv2.COLOR_BGR2GRAY)

    left = adjustGamma(left, gamma=gamma.value)
    right = adjustGamma(right, gamma=gamma.value)
    
    left = cv2.remap(left,Left_Stereo_Map_x,Left_Stereo_Map_y, cv2.INTER_LANCZOS4, cv2.BORDER_CONSTANT, 0)
    right = cv2.remap(right,Right_Stereo_Map_x,Right_Stereo_Map_y, cv2.INTER_LANCZOS4, cv2.BORDER_CONSTANT, 0)

    M = np.float32([[1, 0, shift.value],[0, 1, 0]])
    right = cv2.warpAffine(right, M, (right.shape[1], right.shape[0]))
    
    out = currentL.copy()
    out[:,:,0] = right[:,:]
    out[:,:,1] = right[:,:]    
    out[:,:,2] = left[:,:]

    cropX = 50 + abs(shift.value)
    cropY = 50
    out = out[0+cropY:616-cropY, 0+cropX:820-cropX]
    anaglyph.value=bgr8_to_jpeg(out)
    cv2.imwrite('anaglyph.png', out) 

In [38]:
anaglyph = widgets.Image()
button = widgets.Button(description="Create Anaglyph")
snap = widgets.Button(description="Take photo")
shift = widgets.IntSlider(value=0, min=-100, max=100, step=1, description='shift:')
gamma = widgets.FloatSlider(value=1.0, min=0.0, max=2.0, step=0.05, description='gamma:')

controls = widgets.HBox([gamma, shift, button, snap])
display(widgets.VBox([anaglyph, controls]))
button.on_click(makeAnaglyph)
snap.on_click(snapPhoto)

snapPhoto(None)

In [43]:
cameraL.stop()
cameraR.stop()